# Objetivo del Notebook

El principal objetivo a llevar a cabo en este Jupyter Notebook, no es otro que llevar a cabo el proceso desarrollo del modelo de clasificicion, que aprenda la relacion existente entre las diferentes caracteristicas recogidas en el dataset, y la variable dependiente.

El proceso del desarrollo del modelo recoge los pasos de instanciacion y entrenamiento del modelo, y validacion del mismo una vez ha sido entrenado.

# Importar librerias a utilizar

In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle

In [3]:
# Importo el conjunto de datos desde el archivo .csv
dataset_route = '../../CardioCheck_SVM/data/processed/cardio_processed.csv'
cardio_df = pd.read_csv(dataset_route, index_col = 'id')

# Muestro los primeros 10 ejemplos del DataFrame
cardio_df.head(10)

,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,overweight,cardio
id,,,,,,,,,,,
0,0.436293,1.0,0.105000,0.2500,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.629344,0.0,0.333333,0.5000,1.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.486486,0.0,0.166667,0.0000,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.355212,1.0,0.500000,0.7500,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.339768,0.0,0.105000,0.2825,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.806950,0.0,0.000000,0.2500,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.830116,0.0,0.166667,0.2500,1.0,0.0,0.0,0.0,1.0,1.0,0.0
12,0.880309,1.0,0.166667,0.5000,1.0,1.0,0.0,0.0,1.0,1.0,1.0
13,0.359073,0.0,0.105000,0.0000,0.0,0.0,0.0,0.0,1.0,1.0,0.0


El dataset ya se encuentra preparado para el desarrollo del modelo, por lo que puedo proceder a instanciar el modelo a utilizar.

# Division del conjunto de datos

In [4]:
X = cardio_df.drop(columns = ['cardio'])
y = cardio_df['cardio']

X, y = shuffle(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

In [5]:
X_train.shape

(55204, 10)

In [6]:
X_test.shape

(13801, 10)

In [7]:
y_train.shape

(55204,)

In [8]:
y_test.shape

(13801,)

## Desarrollo del modelo

In [9]:
# Instancio un objeto de la clase SVC
model = SVC()

# Defino un hashmap con los parametros que me gustaria optimizar
parameters = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'decision_function_shape': ['ovo', 'ovr'],
    'degree': [1, 2, 3, 4]
}

# Instancio un objeto con la clase GridSearchCV
grid_cv = GridSearchCV(model, parameters, n_jobs = -1)

# Entreno el modelo
grid_cv.fit(X_train, y_train)

SVC()

In [12]:
# Muestro la configuracion del mejor modelo entrenado
best_params = grid_cv.best_params_
best_model = grid_cv.best_estimator_

print('Parametros escogidos para el mejor modelo:', best_params)
print()
# Muestro el coeficiente de puntuacion R2 del mejor modelo entrenado para el conjunto de testing
best_model.score(X_test, y_test)

Parametros escogidos para el mejor modelo: {'decision_function_shape': 'ovo', 'degree': 1, 'kernel': 'rbf'}



0.723715672777335

In [13]:
# Genero predicciones para el conjunto de testing, con el mejor modelo
y_pred = best_model.predict(X_test)

In [14]:
## Evaluo el rendimiento del modelo representando las predicciones en una matriz de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

class_report = classification_report(y_test, y_pred)
print(class_report)

[[5668 1277]
 [2536 4320]]
              precision    recall  f1-score   support

         0.0       0.69      0.82      0.75      6945
         1.0       0.77      0.63      0.69      6856

    accuracy                           0.72     13801
   macro avg       0.73      0.72      0.72     13801
weighted avg       0.73      0.72      0.72     13801



Se puede apreciar como el modelo lograr de manera aproximada, un 70% de precision al momento de clasificar correctamente cualquiera de las dos posibles clases de la variable dependiente, haciendolo algo peor al momento de clasificar como sano a un determinado paciente dado.

Por el momento, para este conjunto de datos procesado, y tras entrenar el modelo de clasificacion mediante optimizacion de hiperparametros por GridSearchCV, no obtenemos un rendimiento demasiado optimo, por lo que podemos estimar que la modelizacion del conjunto de datos mediante una maquina de soporte vectorial no es el enfoque adecuado.